In [70]:
import pandas as pd
import requests
import time
from time import sleep
from bs4 import BeautifulSoup

# practie scraping 538

538 only has player ratings for the current year. So this will be based off their 2020 rankings

In [71]:
url = 'https://projects.fivethirtyeight.com/2020-nba-player-ratings/'
res = requests.get(url)
res.status_code

200

In [72]:
soup = BeautifulSoup(res.content, 'lxml')

In [73]:
table538 = soup.find('table', {'id' : 'table'})

In [74]:
len(table538.find_all('tr'))

449

Grabbing column names - Some of the labels have a double heading. I'm only grabbing the last column heading row and will rename it later just for the sake of time

In [79]:
table538.find_all('tr')[1]

<tr><th>Player</th><th class="team drop-small" data-tsorter="data-str">Team</th><th class="drop-small" data-tsorter="data-str">Position(s)</th><th class="num drop-small" data-tsorter="data-val">Minutes</th><th class="num lower drop-mid" data-tsorter="data-val">Off.</th><th class="num lower drop-mid" data-tsorter="data-val">Def.</th><th class="num lower drop-mid" data-tsorter="data-val">Total</th><th class="num lower drop-mid" data-tsorter="data-val">Off.</th><th class="num lower drop-mid" data-tsorter="data-val">Def.</th><th class="num lower drop-mid" data-tsorter="data-val">Total</th><th class="num lower" data-tsorter="data-val">Off.</th><th class="num lower" data-tsorter="data-val">Def.</th><th class="num lower" data-tsorter="data-val">Total</th><th class="num sorted" data-tsorter="data-val">WAR</th></tr>

In [95]:
labels

'WAR'

In [76]:
#searching for all Tr in the chart 
#starting with 3rd index because that's where James Harden starts
players = table538.find_all('tr')[2:]

In [100]:
scrape = []
col_names = []
for label in table538.find_all('tr')[1]:
    find = label.text
    col_names.append(find)
for i in range(len(players)):
    look = [details.text for details in players[i].find_all('td')]
    scrape.append(look)

In [101]:
#turning scrape into dataframe
scrape538  = pd.DataFrame(scrape, columns = col_names)

In [102]:
scrape538.head()

,Player,Team,Position(s),Minutes,Off.,Def.,Total,Off.,Def.,Total,Off.,Def.,Total,WAR
0,James Harden,Rockets,"PG, SG",513,+11.7,-0.7,+11.0,+2.5,+10.8,+13.3,+10.9,+0.9,+11.8,3.8
1,LeBron James,Lakers,"PG, SF, PF",489,+8.0,+0.6,+8.6,+14.4,+1.8,+16.3,+9.2,+0.8,+10.0,3.2
2,Luka Doncic,Mavericks,"PG, SG, SF",455,+9.5,+1.1,+10.6,+3.2,-6.7,-3.6,+8.9,+0.1,+9.0,2.7
3,Damian Lillard,Trail Blazers,PG,536,+8.6,-2.1,+6.5,+6.4,-0.5,+6.0,+8.6,-1.9,+6.7,2.6
4,Karl-Anthony Towns,Timberwolves,C,394,+5.5,+4.7,+10.2,+3.8,-1.0,+2.8,+5.5,+4.1,+9.7,2.5


In [104]:
scrape538.columns[1]

'Team'

In [105]:
scrape538.columns = ['Player', 'Team', 'Position(s)', 'Minutes', 'BSR Off.', 'BSR Def.', 'BSR Total',
                    'OOR Off.', 'OOR Def.', 'OOR Total', 'OVR Off.', 'OVR Def.', 'OVR total', 'WAR']

In [ ]:
scrape538.rename()